In [ ]:
import tensorflow as tf
import keras
import os

In [ ]:
gpus = tf.config.list_physical_devices(device_type='GPU')
print(gpus)
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(f"{len(gpus)} Physical GPUs, {len(logical_gpus)} Logical GPUs")
  except RuntimeError as e:
    print(e)

In [ ]:
(training_images, training_labels) , \
(validation_images, validation_labels) = \
tf.keras.datasets.cifar10.load_data()

In [ ]:
def preprocess_image_input(input_images):
  input_images = input_images.astype('float32')
  output_ims = tf.keras.applications.imagenet_utils.preprocess_input(input_images, mode='tf')
  return output_ims

train_X = preprocess_image_input(training_images)
valid_X = preprocess_image_input(validation_images)

In [ ]:
@keras.saving.register_keras_serializable()
class ResizeLayer(tf.keras.layers.Layer):
  def __init__(self, size, method='nearest', **kwargs):
    super(ResizeLayer, self).__init__(**kwargs)
    self.size = size
    self.method = method

  def call(self, inputs):
    return tf.image.resize(inputs, size=self.size, method=self.method)

'''
Feature Extraction is performed by ResNet152 pretrained on imagenet weights. 
Input size is 224 x 224.
'''
def feature_extractor(inputs):
  feature_extractor = tf.keras.applications.resnet.ResNet152(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')(inputs)
  return feature_extractor


'''
Defines final dense layers and subsequent softmax layer for classification.
'''
def classifier(inputs):
  x = tf.keras.layers.GlobalAveragePooling2D()(inputs)
  x = tf.keras.layers.Flatten()(x)
  x = tf.keras.layers.Dense(1024, activation="relu")(x)
  x = tf.keras.layers.Dense(512, activation="relu")(x)
  x = tf.keras.layers.Dense(10, activation="softmax", name="classification")(x)
  return x

'''
Since input image size is (32 x 32), first upsample the image by factor of (7x7) to transform it to (224 x 224)
Connect the feature extraction and "classifier" layers to build the model.
'''
def final_model(inputs):
  resize = ResizeLayer(size=(inputs.shape[1] * 7, inputs.shape[2] * 7))(inputs)
  resnet_feature_extractor = feature_extractor(resize)
  classification_output = classifier(resnet_feature_extractor)
  return classification_output

'''
Define the model and compile it. 
Use Stochastic Gradient Descent as the optimizer.
Use Sparse Categorical CrossEntropy as the loss function.
'''
def define_compile_model():
  inputs = tf.keras.layers.Input(shape=(32,32,3))
  
  classification_output = final_model(inputs) 
  model = tf.keras.Model(inputs=inputs, outputs = classification_output)
 
  model.compile(optimizer='SGD', 
                loss='sparse_categorical_crossentropy',
                metrics = ['accuracy'])
  
  return model


model = define_compile_model()

model.summary()

In [ ]:
EPOCHS = 3

history = model.fit(train_X, 
                    training_labels, 
                    epochs=EPOCHS, 
                    validation_data = (valid_X, validation_labels), 
                    batch_size=64)

In [ ]:
MODEL_PATH = "../models_data/resnet152/"
os.makedirs(MODEL_PATH, exist_ok=True)
model.save(MODEL_PATH+'resnet152_fp32_cifar10.keras')

In [ ]:
model.evaluate(valid_X, validation_labels)
model.evaluate(valid_X[:2000], validation_labels[:2000])

In [ ]:
def representative_dataset():
  for images in tf.data.Dataset.from_tensor_slices(train_X).batch(1).take(1000):
    yield [images]

loaded_model = keras.saving.load_model("../models_data/resnet152/resnet152_fp32_cifar10.keras")
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT] 
converter.representative_dataset = representative_dataset

converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
tflite_quant_model = converter.convert()
with open('../models_data/resnet152/resnet152_uint8_cifar10.tflite', 'wb') as f:
  f.write(tflite_quant_model)

converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
tflite_quant_model = converter.convert()
with open('../models_data/resnet152/resnet152_int8_cifar10.tflite', 'wb') as f:
  f.write(tflite_quant_model)